<a href="https://colab.research.google.com/github/matheusalvesmartins/datascience_ChurnPrediction_SaaS-B2B/blob/main/ChurnScore_e_Predi%C3%A7%C3%A3o_de_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='#330870'> **Inimigo do Churn: uma análise exploratória by Matt Damon**

# <font color='#843CFF'><b>INTRODUÇÃO

<font color='#330870'> **O problema de negócio**

<font color='#330870'> Em empresas SaaS B2B, especialmente em plataformas de CRM, o churn de clientes representa um risco significativo tanto para a previsibilidade de receita quanto para a eficiência operacional das equipes de Customer Success. O cancelamento raramente ocorre de forma abrupta; em geral, ele é precedido por sinais comportamentais como redução de uso do sistema, aumento de chamados de suporte, insatisfação expressa em interações com a equipe de CS ou queda em métricas de percepção de valor. No entanto, na ausência de mecanismos estruturados de priorização, esses sinais tendem a ser analisados de forma reativa e subjetiva, dificultando a alocação eficiente do esforço do time de Customer Success nos clientes com maior risco de churn.

<font color='#330870'> **Os dados disponíveis**

<font color='#330870'> Para enfrentar esse problema, este estudo utiliza dados consolidados em um data lake no Google BigQuery, integrando múltiplas fontes operacionais do ecossistema do CRM. Estão disponíveis informações cadastrais dos clientes, dados financeiros (MRR), métricas de engajamento com o produto, registros de tickets de suporte, chamados de produto, histórico de reuniões com Customer Success — incluindo extração de sinais qualitativos a partir de transcrições — e métricas de satisfação do cliente, como NPS. Esses dados são agregados em nível de cliente e organizados de forma temporal, permitindo a análise de tendências e variações comportamentais ao longo do tempo, fundamentais para a construção de indicadores de risco de churn.

# <font color='#843CFF'><b>OBJETIVO

<font color='#330870'> **Objetivo Geral**

<font color='#330870'> Desenvolver um churn score baseado em dados comportamentais, operacionais e de satisfação do cliente, capaz de identificar e priorizar clientes com maior risco de churn, apoiando a atuação proativa da equipe de Customer Success na retenção de clientes em um contexto SaaS B2B de CRM.

<font color='#330870'> **Objetivo Específico**

* <font color='#330870'>Estruturar uma base analítica integrada, consolidando múltiplas fontes de dados em uma visão única por cliente.

* <font color='#330870'>Definir e calcular métricas de engajamento, suporte e relacionamento com Customer Success, com foco em tendências e variações temporais.

* <font color='#330870'>Construir um churn score explicável, baseado em regras e pesos interpretáveis, adequado para uso operacional.

* <font color='#330870'>Classificar clientes em faixas de risco de churn, viabilizando a priorização de ações preventivas pela equipe de Customer Success.

* <font color='#330870'>Estabelecer uma base metodológica que permita a evolução futura do churn score para modelos preditivos supervisionados.

# <font color='#843CFF'><b>DESENVOLVIMENTO

## 00  <font color='#843CFF'><b>PREPARAÇÃO DO AMBIENTE

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import auth

from google.cloud import bigquery
import pandas as pd

In [ ]:
auth.authenticate_user()

In [ ]:
!pip install --quiet google-cloud-bigquery pandas-gbq

## 01 <font color='#843CFF'><b>CARREGAMENTO DOS DADOS

In [ ]:
# Lendo os dados do dataset já preparado no GoogleBigQuery
client = bigquery.Client(project="ploomes-441013")

query = """
SELECT *
FROM `ploomes-441013.views_analytics.churn_score_features_v1`
"""

df = client.query(query).to_dataframe()

In [ ]:
# exibindo o cabecalho dos dados
df.head()

In [ ]:
# verificando a dimensão do dataset
df.shape

In [ ]:
# verificando os tipos de dados e valores nulos
df.info()

## 02 <font color='#843CFF'><b>EXPLORAÇÃO DOS DADOS

In [ ]:
# definindo as variáveis "variaveis_categoricas" e "variaveis_numericas" a partir do tipo de campo.
# se for float64 é numérica, se não é categórica
variaveis_categoricas = [var for var in df.columns if df[var].dtype=='O']
variaveis_numericas = [var for var in df.columns if df[var].dtype!='O']

In [ ]:
# criando looping para calcular a distribuição percentual para cada variável categórica
for coluna in variaveis_categoricas:
  print(f'Distribuição percentual para {coluna}')
  print(df[coluna].value_counts(normalize=True))
  print()

In [ ]:
# criando looping para plotar gráfico de barras horizontais da distribuição das variáveis categóricas
for coluna in variaveis_categoricas:
  df[coluna].value_counts().sort_values(ascending=True).plot(kind='barh', figsize=(8,12), rot=0, color='purple')
  plt.xlabel('Frequência')
  plt.ylabel(coluna)
  plt.title(f'Distribuição de {coluna}')
  plt.show()
  print()

## 03 <font color='#843CFF'><b>CHURN SCORE

<font color='#330870'> O **churn score** é um indicador composto que tem como objetivo **estimar o risco relativo de cancelamento de um cliente** a partir da análise de sinais comportamentais, operacionais e de percepção de valor observados ao longo do tempo. A metodologia adotada neste trabalho baseia-se em uma abordagem heurística e explicável, inspirada em modelos de lead scoring, na qual diferentes dimensões do relacionamento do cliente com o produto e com a empresa — como engajamento com o sistema, volume e tendência de chamados de suporte, interações com a equipe de Customer Success e métricas de satisfação (NPS) — são transformadas em regras de pontuação. Cada regra reflete um padrão historicamente associado a maior risco de churn, atribuindo pesos proporcionais à severidade do sinal identificado. O score final resulta da soma dessas pontuações, normalizado em uma escala de 0 a 100, permitindo a classificação dos clientes em faixas de risco e viabilizando sua utilização tanto como ferramenta operacional de priorização quanto como base para evoluções futuras em modelos preditivos supervisionados.


> A abordagem de churn score adotada neste estudo está alinhada com a literatura recente sobre predição de churn em contextos B2B e SaaS, que aponta o comportamento de uso do produto, o histórico de interações de suporte e a percepção de valor do cliente como antecedentes importantes do cancelamento (RAMIREZ et al., 2024; SANCHES; POSSEBOM; AYLON, 2025). Estudos recentes destacam que o churn, especialmente em modelos SaaS, raramente é um evento abrupto, sendo precedido por mudanças graduais em padrões de engajamento, aumento de atrito operacional e deterioração da satisfação do cliente, o que justifica o uso de métricas agregadas e baseadas em tendências temporais, em vez de indicadores pontuais (MARTIN, 2024). Além disso, trabalhos contemporâneos reforçam a importância de modelos explicáveis e orientados a risco, capazes de apoiar decisões operacionais, como a priorização de clientes pela equipe de Customer Success, mesmo antes da aplicação de modelos preditivos supervisionados (KIMITEI, 2025; DE ALWIS et al., 2025). Nesse contexto, o churn score funciona como uma medida composta de risco, integrando sinais quantitativos e qualitativos para fornecer uma estimativa interpretável do risco relativo de churn, consistente com práticas recomendadas na literatura recente sobre churn em ambientes B2B/SaaS.

---

> RAMIREZ, J. S. et al. Incorporating usage data for B2B churn prediction modeling. Journal of Business Analytics, 2024. Disponível em: https://www.sciencedirect.com/science/article/abs/pii/S0019850124000865
. Acesso em: 29 dez. 2025.

> MARTIN, E. Predicting Customer Churn at a SaaS B2B Company. Dissertação (MSc) — [instituição], 2024. Disponível em: https://www.diva-portal.org/smash/get/diva2%3A1901072/FULLTEXT01.pdf
. Acesso em: 29 dez. 2025.

> SANCHES, H. E.; POSSEBOM, A. T.; AYLON, L. B. R. Churn prediction for SaaS company with machine learning. International Journal of Data Science and Analytics, 2025. Disponível em: https://revistas.usp.br/rai/article/download/239160/215694/771744
. Acesso em: 29 dez. 2025.

> KIMITEI, S. Predictability & explainability of survival analysis in churn prediction. Journal of Risk Modeling and Interpretation, 2025. Disponível em: https://link.springer.com/article/10.1057/s41270-025-00450-2
. Acesso em: 29 dez. 2025.

> DE ALWIS, S. et al. Explainability, risk modeling, and segmentation for churn prediction. arXiv preprint, 2025. Disponível em: https://arxiv.org/pdf/2510.11604
. Acesso em: 29 dez. 2025.


### 03-01 <font color='#330870'><b> Calculando o Churn Score

<font color='#330870'> Calculando o ChurnScore

In [ ]:
df_backup = df.copy()

In [ ]:
#df = df_backup.copy()

In [ ]:
df_backup.Billing_PaganteAtivo.value_counts()

In [ ]:
import pandas as pd
import numpy as np

# ==========================================================
# 0) (IMPORTANTE) Remover qualquer resíduo de cálculo anterior
# ==========================================================
# Garante que não existe "acúmulo" por reexecuções do notebook
cols_reset = [
    "churn_score_engajamento",
    "churn_score_suporte",
    "churn_score_satisfacao",
    "churn_score",
    "churn_risk",
    "flag_nps_missing",
    "flag_qoq_users_missing",
    "flag_tickets_qoq_missing",
]
for c in cols_reset:
    if c in df.columns:
        df.drop(columns=[c], inplace=True)

# =========================
# 1) Tipagem numérica (sem destruir campos categóricos/booleanos)
# =========================
non_numeric_cols = [
    "Nome",
    "CS_EstagioJornada",
    "ADM_TipoImplementacao",
    "MOC",
    "ADM_ClassificacaoPosVendas",
    "Status",
    "DataCriacao_Date",
    "ADM_DataAtivacao_Date",
    "Billing_PaganteAtivo",  # <<< importante: não converter aqui
]

numeric_cols = df.columns.difference(non_numeric_cols)
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Normalizar Billing_PaganteAtivo para 0/1 (mantém rastreável e modelável)
if "Billing_PaganteAtivo" in df.columns:
    s = df["Billing_PaganteAtivo"]

    # se já vier booleano, converte direto
    if s.dtype == bool:
        df["Billing_PaganteAtivo"] = s.astype(int)
    else:
        # trata strings comuns
        norm = s.astype(str).str.strip().str.lower()
        df["Billing_PaganteAtivo"] = np.where(
            norm.isin(["1", "true", "t", "sim", "s", "yes", "y", "ativo", "pagante", "pago"]),
            1,
            np.where(
                norm.isin(["0", "false", "f", "nao", "não", "n", "no", "inativo", "free", "trial", "gratuito"]),
                0,
                np.nan  # caso desconhecido
            )
        )


# =========================
# 2) Flags auxiliares (missing != neutro)
# =========================
df["flag_nps_missing"] = df["NPS_AvgLastYear"].isna().astype(int)
df["flag_qoq_users_missing"] = df["Engage_QOQ_UsersAtivos"].isna().astype(int)
df["flag_tickets_qoq_missing"] = df["Tickets_QOQ_TicketsMonthly"].isna().astype(int)

# =========================
# 3) Inicializar scores parciais + total (ZERANDO DE FATO)
# =========================
df["churn_score_engajamento"] = 0
df["churn_score_suporte"] = 0
df["churn_score_satisfacao"] = 0
df["churn_score"] = 0

# ==========================================================
# 4) Engajamento (rebalanceado, com desvio padrão como tolerância)
# ==========================================================
def rel_tol(avg_col, std_col):
    avg = df[avg_col].astype(float)
    std = df[std_col].astype(float)
    tol = std / avg.replace(0, np.nan)
    return tol.replace([np.inf, -np.inf], np.nan)

def drop_from_qoq(qoq_col):
    qoq = df[qoq_col].astype(float)
    drop = (1 - qoq).clip(lower=0)  # só penaliza queda (QoQ<1)
    return drop.replace([np.inf, -np.inf], np.nan)

# -------------------------
# 4.1 UsersAtivos
# -------------------------
users_drop = drop_from_qoq("Engage_QOQ_UsersAtivos")
users_tol  = rel_tol("Engage_Avg_UsersAtivos", "Engage_StDev_UsersAtivos")
users_excess = (users_drop - users_tol).where(users_tol.notna() & users_drop.notna())

df.loc[(users_excess > 0) & (users_excess <= 0.20), "churn_score_engajamento"] += 5
df.loc[(users_excess > 0.20), "churn_score_engajamento"] += 10

# Missing Users QoQ: não pontua (explicitamente)
df.loc[df["flag_qoq_users_missing"] == 1, "churn_score_engajamento"] += 0

# -------------------------
# 4.2 DealsMovimentados
# -------------------------
deals_drop = drop_from_qoq("Engage_QOQ_DealsMovimentados")
deals_tol  = rel_tol("Engage_Avg_DealsMovimentados", "Engage_StDev_DealsMovimentados")
deals_excess = (deals_drop - deals_tol).where(deals_tol.notna() & deals_drop.notna())

df.loc[(deals_excess > 0) & (deals_excess <= 0.30), "churn_score_engajamento"] += 5
df.loc[(deals_excess > 0.30), "churn_score_engajamento"] += 10

# -------------------------
# 4.3 PropostasCriadas
# -------------------------
props_drop = drop_from_qoq("Engage_QOQ_PropostasCriadas")
props_tol  = rel_tol("Engage_Avg_PropostasCriadas", "Engage_StDev_PropostasCriadas")
props_excess = (props_drop - props_tol).where(props_tol.notna() & props_drop.notna())

df.loc[(props_excess > 0) & (props_excess <= 0.30), "churn_score_engajamento"] += 5
df.loc[(props_excess > 0.30), "churn_score_engajamento"] += 10

# -------------------------
# 4.4 TasksFinalizadas
# -------------------------
tasks_drop = drop_from_qoq("Engage_QOQ_TasksFinalizadas")
tasks_tol  = rel_tol("Engage_Avg_TasksFinalizadas", "Engage_StDev_TasksFinalizadas")
tasks_excess = (tasks_drop - tasks_tol).where(tasks_tol.notna() & tasks_drop.notna())

df.loc[(tasks_excess > 0) & (tasks_excess <= 0.30), "churn_score_engajamento"] += 5
df.loc[(tasks_excess > 0.30), "churn_score_engajamento"] += 10

# ==========================================================
# 5) Suporte / Produto
# ==========================================================
df.loc[df["Tickets_QOQ_TicketsMonthly"] > 1.5, "churn_score_suporte"] += 15
df.loc[
    (df["Tickets_QOQ_TicketsMonthly"] > 1.2) &
    (df["Tickets_QOQ_TicketsMonthly"] <= 1.5),
    "churn_score_suporte"
] += 8

df.loc[df["Chamados_QOQ_ChamadosMonthly"] > 1.3, "churn_score_suporte"] += 10

# Missing tickets QoQ: não pontua
df.loc[df["flag_tickets_qoq_missing"] == 1, "churn_score_suporte"] += 0

# ==========================================================
# 6) Satisfação (com override opcional)
# ==========================================================
df.loc[df["ReuniaoCS_MentionChurn"] > 0, "churn_score_satisfacao"] += 30

df.loc[
    df["ReuniaoCS_FeedbackNegativo"] > df["ReuniaoCS_FeedbackPositivo"],
    "churn_score_satisfacao"
] += 10

df.loc[df["NPS_AvgLastYear"] < 7, "churn_score_satisfacao"] += 5
df.loc[df["NPS_QOQ"] < 1, "churn_score_satisfacao"] += 5

# Missing NPS: não pontua
df.loc[df["flag_nps_missing"] == 1, "churn_score_satisfacao"] += 0

# ==========================================================
# 7) Score total
# ==========================================================
df["churn_score"] = (
    df["churn_score_engajamento"] +
    df["churn_score_suporte"] +
    df["churn_score_satisfacao"]
).clip(0, 100)

# ==========================================================
# 8) churn_risk RECOMPUTADO do ZERO (sobrescreve sempre)
# ==========================================================
def churn_risk(score):
    if score >= 50:
        return "Alto"
    elif score >= 30:
        return "Médio"
    else:
        return "Baixo"

df["churn_risk"] = df["churn_score"].apply(churn_risk)

# =========================
# 10) Quick view
# =========================
df.sort_values("churn_score", ascending=False).head(15)[
    [
        "Nome",
        "churn_score",
        "churn_risk",
        "churn_score_engajamento",
        "churn_score_suporte",
        "churn_score_satisfacao",
        "Billing_PaganteAtivo",
        "Engage_QOQ_UsersAtivos",
        "Engage_StDev_UsersAtivos",
        "Engage_QOQ_DealsMovimentados",
        "Engage_StDev_DealsMovimentados",
        "Engage_QOQ_PropostasCriadas",
        "Engage_StDev_PropostasCriadas",
        "Engage_QOQ_TasksFinalizadas",
        "Engage_StDev_TasksFinalizadas",
        "Tickets_QOQ_TicketsMonthly",
        "Chamados_QOQ_ChamadosMonthly",
        "ReuniaoCS_MentionChurn",
        "NPS_AvgLastYear",
        "NPS_QOQ"
    ]
]


In [ ]:
df.info()

<font color='#330870'> **Componentes do Churn Score.**

<font color='#330870'>O churn score proposto é composto por três dimensões complementares.

* <font color='#330870'>**Engajamento**: avalia a variação do uso do produto ao longo do tempo, considerando a quantidade de usuários ativos, o volume de negócios (deals) movimentados, a quantidade de propostas criadas e o número de tarefas finalizadas, sempre comparando a variação entre trimestres com a volatilidade histórica de cada métrica para evitar penalizações por oscilações naturais.
* <font color='#330870'>**Suporte**: mede o nível de atrito operacional a partir da variação na abertura de tickets de suporte e chamados de produto, comparando o trimestre atual com o trimestre anterior, sob a premissa de que aumentos anormais indicam dificuldades recorrentes ou falhas de adoção.
* <font color='#330870'>**Satisfação**: captura sinais de percepção de valor e risco explícito, combinando a variação do NPS ao longo do tempo com evidências qualitativas extraídas das reuniões de Customer Success, como menções diretas a churn e predominância de feedbacks negativos, permitindo identificar riscos que não são necessariamente refletidos apenas em métricas de uso ou suporte.

### 03-02 <font color='#330870'><b> Análise do Churn Score

In [ ]:
## pegando dados para plotagem do gráfico


#### Churn Score com toda a base de clientes

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# -------------------------
# 0) Garantir tipos
# -------------------------
df["CS_MensalidadeAtual_num"] = pd.to_numeric(df["CS_MensalidadeAtual"], errors="coerce").fillna(0)
df["churn_score"] = pd.to_numeric(df["churn_score"], errors="coerce").fillna(0)
df["churn_score_engajamento"] = pd.to_numeric(df["churn_score_engajamento"], errors="coerce").fillna(0)
df["churn_score_suporte"] = pd.to_numeric(df["churn_score_suporte"], errors="coerce").fillna(0)
df["churn_score_satisfacao"] = pd.to_numeric(df["churn_score_satisfacao"], errors="coerce").fillna(0)

# -------------------------
# 1) Percentuais por cliente (componentes como % do total)
# -------------------------
total = df["churn_score"].replace(0, np.nan)

df["eng_pct"] = (df["churn_score_engajamento"] / total) * 100
df["sup_pct"] = (df["churn_score_suporte"] / total) * 100
df["sat_pct"] = (df["churn_score_satisfacao"] / total) * 100

df[["eng_pct", "sup_pct", "sat_pct"]] = (
    df[["eng_pct", "sup_pct", "sat_pct"]].fillna(0).clip(0, 100)
)

# -------------------------
# 2) Arrays do gráfico (1 trace único = hover sempre correto)
# -------------------------
x = df["churn_score_engajamento"].to_numpy()
y = df["churn_score_suporte"].to_numpy()
z = df["churn_score_satisfacao"].to_numpy()

x_max = float(np.nanmax(x)) if len(x) else 0.0
y_max = float(np.nanmax(y)) if len(y) else 0.0
z_max = float(np.nanmax(z)) if len(z) else 0.0

# Cores por ponto (paleta fixa)
color_map = {"Alto": "red", "Médio": "yellow", "Baixo": "green"}
point_colors = df["churn_risk"].map(color_map).fillna("gray").to_numpy()

# 1ª linha do hover: "PartnersId - Nome"
df["hover_id_nome"] = df["PartnersId"].astype(str) + " - " + df["Nome"].astype(str)

# customdata por ponto (linha-a-linha)
customdata = np.column_stack([
    df["hover_id_nome"].fillna("").to_numpy(),        # 0 (PartnersId - Nome)
    df["CS_MensalidadeAtual_num"].to_numpy(),         # 1
    df["churn_score"].to_numpy(),                     # 2

    df["churn_score_engajamento"].to_numpy(),         # 3
    df["eng_pct"].to_numpy(),                         # 4

    df["churn_score_suporte"].to_numpy(),             # 5
    df["sup_pct"].to_numpy(),                         # 6

    df["churn_score_satisfacao"].to_numpy(),          # 7
    df["sat_pct"].to_numpy(),                         # 8
])

# -------------------------
# 3) Figura
# -------------------------
n_clients = int(df.shape[0])
title_text = (
    "Distribuição 3D dos clientes por componentes do Churn Score"
    f"<br><sup>Total de clientes representados = {n_clients}</sup>"
)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode="markers",
            marker=dict(size=5, opacity=0.85, color=point_colors),
            customdata=customdata,
            hovertemplate=(
                "<b>%{customdata[0]}</b><br>"
                "Mensalidade (CS): R$ %{customdata[1]:,.2f}<br><br>"
                "<b>Churn Score (Total):</b> %{customdata[2]:.1f}%<br><br>"
                "<b>Componentes (pontos e % do total):</b><br>"
                "Engajamento: %{customdata[3]:.0f} pts (%{customdata[4]:.1f}%)<br>"
                "Suporte: %{customdata[5]:.0f} pts (%{customdata[6]:.1f}%)<br>"
                "Satisfação: %{customdata[7]:.0f} pts (%{customdata[8]:.1f}%)<br>"
                "<extra></extra>"
            )
        )
    ]
)

fig.update_layout(
    title=title_text,
    height=740,
    scene=dict(
        xaxis=dict(title="Engajamento (pontos)", range=[0, x_max]),
        yaxis=dict(title="Suporte (pontos)", range=[0, y_max]),
        zaxis=dict(title="Satisfação (pontos)", range=[0, z_max]),
        aspectmode="cube",
    ),
)

fig.show()


In [ ]:
### DOWNLOAD DO HTML DO GRÁFICO 3D
import plotly.io as pio

pio.write_html(
    fig,
    file="churn_3d.html",
    include_plotlyjs=True,
    full_html=True,
    auto_open=False
)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# =========================
# Parâmetros
# =========================
TOP_N = 30  # <<< ajuste manualmente aqui
color_map = {"Alto": "red", "Médio": "yellow", "Baixo": "green"}

# =========================
# Preparar dados
# =========================
bar = df.copy()

# Tipagem
bar["CS_MensalidadeAtual_num"] = pd.to_numeric(bar["CS_MensalidadeAtual"], errors="coerce").fillna(0)
bar["churn_score"] = pd.to_numeric(bar["churn_score"], errors="coerce").fillna(0)
bar["churn_score_engajamento"] = pd.to_numeric(bar["churn_score_engajamento"], errors="coerce").fillna(0)
bar["churn_score_suporte"] = pd.to_numeric(bar["churn_score_suporte"], errors="coerce").fillna(0)
bar["churn_score_satisfacao"] = pd.to_numeric(bar["churn_score_satisfacao"], errors="coerce").fillna(0)

bar["label_cliente"] = bar["PartnersId"].astype(str) + " - " + bar["Nome"].astype(str)

# Percentuais por cliente
total = bar["churn_score"].replace(0, np.nan)
bar["eng_pct"] = (bar["churn_score_engajamento"] / total) * 100
bar["sup_pct"] = (bar["churn_score_suporte"] / total) * 100
bar["sat_pct"] = (bar["churn_score_satisfacao"] / total) * 100
bar[["eng_pct", "sup_pct", "sat_pct"]] = bar[["eng_pct", "sup_pct", "sat_pct"]].fillna(0).clip(0, 100)

# Top N
bar = bar.sort_values("churn_score", ascending=False).head(TOP_N).copy()

# =========================
# Gráfico de barras horizontal
# =========================
fig = px.bar(
    bar,
    x="churn_score",
    y="label_cliente",
    orientation="h",
    color="churn_risk",
    color_discrete_map=color_map,
    text=bar["churn_score"].round(1),  # <<< rótulo do dado
    title=f"Top {len(bar)} clientes por Churn Score"
)

# Rótulos fora da barra
fig.update_traces(
    textposition="outside",
    cliponaxis=False,  # <<< evita cortar o texto
    hovertemplate=(
        "<b>%{y}</b><br>"
        "Mensalidade (CS): R$ %{customdata[0]:,.2f}<br><br>"
        "<b>Churn Score (Total):</b> %{customdata[1]:.1f}%<br><br>"
        "<b>Componentes (pontos e % do total):</b><br>"
        "Engajamento: %{customdata[2]:.0f} pts (%{customdata[3]:.1f}%)<br>"
        "Suporte: %{customdata[4]:.0f} pts (%{customdata[5]:.1f}%)<br>"
        "Satisfação: %{customdata[6]:.0f} pts (%{customdata[7]:.1f}%)<br>"
        "<extra></extra>"
    ),
    customdata=np.column_stack([
        bar["CS_MensalidadeAtual_num"].to_numpy(),      # 0
        bar["churn_score"].to_numpy(),                  # 1
        bar["churn_score_engajamento"].to_numpy(),      # 2
        bar["eng_pct"].to_numpy(),                      # 3
        bar["churn_score_suporte"].to_numpy(),          # 4
        bar["sup_pct"].to_numpy(),                      # 5
        bar["churn_score_satisfacao"].to_numpy(),       # 6
        bar["sat_pct"].to_numpy(),                      # 7
    ])
)

# Layout final
fig.update_layout(
    height=max(520, 22 * len(bar)),   # responsivo ao TOP_N
    yaxis=dict(title="Cliente", autorange="reversed"),
    xaxis=dict(title="Churn Score (Total, %)", range=[0, 105]),  # espaço p/ rótulo externo
    legend_title_text="Risco de Churn",
    margin=dict(l=40, r=40, t=70, b=40),
)

fig.show()

In [ ]:
### exibindo valores de billing_pagante ativo
df.Billing_PaganteAtivo.value_counts()

#### Churn Score com a base de clientes ATIVOS

In [ ]:
### VERSÃO COM BASE NO DATASET DE TREINAMENTO

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# =========================
# Parâmetros
# =========================
TOP_N = 30  # <<< ajuste manualmente aqui
color_map = {"Alto": "red", "Médio": "yellow", "Baixo": "green"}

# =========================
# Preparar dados
# =========================
# filtrando billing_pagante ativo
bar = df[df['Billing_PaganteAtivo'] == 1].copy()

# Tipagem
bar["CS_MensalidadeAtual_num"] = pd.to_numeric(bar["CS_MensalidadeAtual"], errors="coerce").fillna(0)
bar["churn_score"] = pd.to_numeric(bar["churn_score"], errors="coerce").fillna(0)
bar["churn_score_engajamento"] = pd.to_numeric(bar["churn_score_engajamento"], errors="coerce").fillna(0)
bar["churn_score_suporte"] = pd.to_numeric(bar["churn_score_suporte"], errors="coerce").fillna(0)
bar["churn_score_satisfacao"] = pd.to_numeric(bar["churn_score_satisfacao"], errors="coerce").fillna(0)

bar["label_cliente"] = bar["PartnersId"].astype(str) + " - " + bar["Nome"].astype(str)

# Percentuais por cliente
total = bar["churn_score"].replace(0, np.nan)
bar["eng_pct"] = (bar["churn_score_engajamento"] / total) * 100
bar["sup_pct"] = (bar["churn_score_suporte"] / total) * 100
bar["sat_pct"] = (bar["churn_score_satisfacao"] / total) * 100
bar[["eng_pct", "sup_pct", "sat_pct"]] = bar[["eng_pct", "sup_pct", "sat_pct"]].fillna(0).clip(0, 100)

# Top N
bar = bar.sort_values("churn_score", ascending=False).head(TOP_N).copy()

# =========================
# Gráfico de barras horizontal
# =========================
fig = px.bar(
    bar,
    x="churn_score",
    y="label_cliente",
    orientation="h",
    color="churn_risk",
    color_discrete_map=color_map,
    text=bar["churn_score"].round(1),  # <<< rótulo do dado
    title=f"Top {len(bar)} clientes por Churn Score"
)

# Rótulos fora da barra
fig.update_traces(
    textposition="outside",
    cliponaxis=False,  # <<< evita cortar o texto
    hovertemplate=(
        "<b>%{y}</b><br>"
        "Mensalidade (CS): R$ %{customdata[0]:,.2f}<br><br>"
        "<b>Churn Score (Total):</b> %{customdata[1]:.1f}%<br><br>"
        "<b>Componentes (pontos e % do total):</b><br>"
        "Engajamento: %{customdata[2]:.0f} pts (%{customdata[3]:.1f}%)<br>"
        "Suporte: %{customdata[4]:.0f} pts (%{customdata[5]:.1f}%)<br>"
        "Satisfação: %{customdata[6]:.0f} pts (%{customdata[7]:.1f}%)<br>"
        "<extra></extra>"
    ),
    customdata=np.column_stack([
        bar["CS_MensalidadeAtual_num"].to_numpy(),      # 0
        bar["churn_score"].to_numpy(),                  # 1
        bar["churn_score_engajamento"].to_numpy(),      # 2
        bar["eng_pct"].to_numpy(),                      # 3
        bar["churn_score_suporte"].to_numpy(),          # 4
        bar["sup_pct"].to_numpy(),                      # 5
        bar["churn_score_satisfacao"].to_numpy(),       # 6
        bar["sat_pct"].to_numpy(),                      # 7
    ])
)

# Layout final
fig.update_layout(
    height=max(520, 22 * len(bar)),   # responsivo ao TOP_N
    yaxis=dict(title="Cliente", autorange="reversed"),
    xaxis=dict(title="Churn Score (Total, %)", range=[0, 105]),  # espaço p/ rótulo externo
    legend_title_text="Risco de Churn",
    margin=dict(l=40, r=40, t=70, b=40),
)

fig.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# criando dataFrame dos clientes ativos
df2 = df[df['Billing_PaganteAtivo'] == 1]

# -------------------------
# 0) Garantir tipos
# -------------------------
df2["CS_MensalidadeAtual_num"] = pd.to_numeric(df2["CS_MensalidadeAtual"], errors="coerce").fillna(0)
df2["churn_score"] = pd.to_numeric(df2["churn_score"], errors="coerce").fillna(0)
df2["churn_score_engajamento"] = pd.to_numeric(df2["churn_score_engajamento"], errors="coerce").fillna(0)
df2["churn_score_suporte"] = pd.to_numeric(df2["churn_score_suporte"], errors="coerce").fillna(0)
df2["churn_score_satisfacao"] = pd.to_numeric(df2["churn_score_satisfacao"], errors="coerce").fillna(0)

# -------------------------
# 1) Percentuais por cliente (componentes como % do total)
# -------------------------
total = df2["churn_score"].replace(0, np.nan)

df2["eng_pct"] = (df2["churn_score_engajamento"] / total) * 100
df2["sup_pct"] = (df2["churn_score_suporte"] / total) * 100
df2["sat_pct"] = (df2["churn_score_satisfacao"] / total) * 100

df2[["eng_pct", "sup_pct", "sat_pct"]] = (
    df2[["eng_pct", "sup_pct", "sat_pct"]].fillna(0).clip(0, 100)
)

# -------------------------
# 2) Arrays do gráfico (1 trace único = hover sempre correto)
# -------------------------
x = df2["churn_score_engajamento"].to_numpy()
y = df2["churn_score_suporte"].to_numpy()
z = df2["churn_score_satisfacao"].to_numpy()

x_max = float(np.nanmax(x)) if len(x) else 0.0
y_max = float(np.nanmax(y)) if len(y) else 0.0
z_max = float(np.nanmax(z)) if len(z) else 0.0

# Cores por ponto (paleta fixa)
color_map = {"Alto": "red", "Médio": "yellow", "Baixo": "green"}
point_colors = df2["churn_risk"].map(color_map).fillna("gray").to_numpy()

# 1ª linha do hover: "PartnersId - Nome"
df2["hover_id_nome"] = df2["PartnersId"].astype(str) + " - " + df2["Nome"].astype(str)

# customdata por ponto (linha-a-linha)
customdata = np.column_stack([
    df2["hover_id_nome"].fillna("").to_numpy(),        # 0 (PartnersId - Nome)
    df2["CS_MensalidadeAtual_num"].to_numpy(),         # 1
    df2["churn_score"].to_numpy(),                     # 2

    df2["churn_score_engajamento"].to_numpy(),         # 3
    df2["eng_pct"].to_numpy(),                         # 4

    df2["churn_score_suporte"].to_numpy(),             # 5
    df2["sup_pct"].to_numpy(),                         # 6

    df2["churn_score_satisfacao"].to_numpy(),          # 7
    df2["sat_pct"].to_numpy(),                         # 8
])

# -------------------------
# 3) Figura
# -------------------------
n_clients = int(df2.shape[0])
title_text = (
    "Distribuição 3D dos clientes por componentes do Churn Score"
    f"<br><sup>Total de clientes representados = {n_clients}</sup>"
)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode="markers",
            marker=dict(size=5, opacity=0.85, color=point_colors),
            customdata=customdata,
            hovertemplate=(
                "<b>%{customdata[0]}</b><br>"
                "Mensalidade (CS): R$ %{customdata[1]:,.2f}<br><br>"
                "<b>Churn Score (Total):</b> %{customdata[2]:.1f}%<br><br>"
                "<b>Componentes (pontos e % do total):</b><br>"
                "Engajamento: %{customdata[3]:.0f} pts (%{customdata[4]:.1f}%)<br>"
                "Suporte: %{customdata[5]:.0f} pts (%{customdata[6]:.1f}%)<br>"
                "Satisfação: %{customdata[7]:.0f} pts (%{customdata[8]:.1f}%)<br>"
                "<extra></extra>"
            )
        )
    ]
)

fig.update_layout(
    title=title_text,
    height=740,
    scene=dict(
        xaxis=dict(title="Engajamento (pontos)", range=[0, x_max]),
        yaxis=dict(title="Suporte (pontos)", range=[0, y_max]),
        zaxis=dict(title="Satisfação (pontos)", range=[0, z_max]),
        aspectmode="cube",
    ),
)

fig.show()


## 04 <font color='#843CFF'><b>PREDIÇÃO DE CHURN

<font color='#330870'> Agora com um dataset já tratado e com Churn score, vamos consultar dados de clientes que churnaram anteriormente e assim vamos conseguir ter a label de clientes churnados para mesclar com o nosso dataset.

<font color='#330870'> A partir desse ponto, poderemos treinar um modelo de machine learning para predição de churn.

### 04-00 <font color='#330870'><b> Obtendo a label de Churn

In [ ]:
# Lendo os dados do dataset clientes_churn
client = bigquery.Client(project="ploomes-441013")

query = """
SELECT *
FROM `ploomes-441013.ploomes.clientes_churn`
"""

df_label_churn = client.query(query).to_dataframe()

In [ ]:
df_label_churn.head()

In [ ]:
df_label_churn.info()

In [ ]:
df_label_churn_backup = df_label_churn.copy()

In [ ]:
#df_label_churn = df_label_churn_backup.copy()

In [ ]:
# tratando df_label_churn e mantendo somente os campos PloomesId, PartnersId, _Billing_|_M_s_de_churn, _Adm_|_Estadia_total_em_meses_
df_label_churn = df_label_churn[['PloomesId','_Billing_|_M_s_de_churn', '_Adm_|_Estadia_total_em_meses_', 'PartnersId']]

df_label_churn.head()

In [ ]:
# removendo os valores onde _Billing_|_M_s_de_churn é vazio, nulo ou NaT
df_label_churn = df_label_churn.dropna(subset=['_Billing_|_M_s_de_churn'])
df_label_churn.head()

In [ ]:
df_label_churn.info()

In [ ]:
# fazendo a junção de df_label_churn com df em um novo dataframe df_churn
# usando df como dataset principal, trazer os valores de "_Billing_|_M_s_de_churn" e "_Adm_|_Estadia_total_em_meses_" vinculando PloomesId com PloomesId
# se não encontrar o par, deixar os valores vazios
# incluir uma nova coluna chamada "churn" de sim/não. se estiver com o valor de "_Billing_|_M_s_de_churn" preenchida, vai ser True, se estiver sem valor é False
df_churn = pd.merge(df, df_label_churn, left_on='PloomesId', right_on='PloomesId', how='left')
df_churn.head()

In [ ]:
df_churn.info()

In [ ]:
# removendo "PartnersId_y"
df_churn = df_churn.drop(columns=['PartnersId_y'])

# renomeando "_Billing_|_M_s_de_churn" para "data_churn" e "_Adm_|_Estadia_total_em_meses_" para "estadia_meses"
df_churn = df_churn.rename(columns={'_Billing_|_M_s_de_churn': 'data_churn', '_Adm_|_Estadia_total_em_meses_': 'estadia_churn_meses'})

# incluindo a coluna de "churn", é vazio então "True"
df_churn['churn'] = df_churn['data_churn'].notna()

df_churn.head()

In [ ]:
# exibindo a proporção da distribuição pelo valor churn
df_churn['churn'].value_counts(normalize=True)

In [ ]:
# removendo MRR_Atual
df_churn = df_churn.drop(columns=['MRR_Atual'])

In [ ]:
# calculando a data de estadia real hoje com base em na diferença em meses da data hoje(today) pela data do campo "ADM_DataAtivacao_Date"
df_churn['estadia_real_meses'] = (pd.to_datetime('today') - pd.to_datetime(df_churn['ADM_DataAtivacao_Date'])).dt.days // 30

### 04-01 <font color='#330870'><b> Pré-processamento dos dados

<font color='#330870'> Considerando os pontos observados na análise exploratória, identificamos a necessidade de tratar os outliers da variável numérica existente no dataset. Portanto, iniciamos o pré-processamento do dataset **removendo os outliers de [CS_MensalidadeAtual]**.

In [ ]:
## REMOVENDO OUTLIERS

# Definindo uma função para remover Outliers de um dataframe com base em uma variável numérica e opcionalmente uma variável categórica
def remover_outliers_iqr(df, variavel_num, variavel_cat=None):    # define a função com a variavel_cat como opcional
  if variavel_cat is not None:                                 # se a variável categórica for fornecida
    df_variavel_cat = pd.DataFrame()                            # cria um dataframe vazio
    for categoria in df[variavel_cat].unique():                 # para cada valor fornecido na variável categórica
      df_categoria = df[df[variavel_cat] == categoria]            # cria um dataframe com os dados da categoria
      Q1 = df_categoria[variavel_num].quantile(0.25)              # calcula o primeiro quartil
      Q3 = df_categoria[variavel_num].quantile(0.75)              # calcula o terceiro quartil
      IQR = Q3 - Q1                                               # calcula o intervalo interquartil
      limite_inferior = Q1 - 1.5 * IQR                            # calcula o limite inferior
      limite_superior = Q3 + 1.5 * IQR                            # calcula o limite superior
      df_categoria = df_categoria[(df_categoria[variavel_num] >= limite_inferior) & (df_categoria[variavel_num] <= limite_superior)] # remove os outliers
      df_variavel_cat = pd.concat([df_variavel_cat, df_categoria]) # concatena os dataframes
    return df_variavel_cat                                      # retorna o dataframe com os outliers removidos
  else:                                                         # se a variável categórica não for fornecida
    Q1 = df[variavel_num].quantile(0.25)                          # calcula o primeiro quartil
    Q3 = df[variavel_num].quantile(0.75)                          # calcula o terceiro quartil
    IQR = Q3 - Q1                                                 # calcula o intervalo interquartil
    limite_inferior = Q1 - 1.5 * IQR                              # calcula o limite inferior
    limite_superior = Q3 + 1.5 * IQR                              # calcula o limite superior
    df = df[(df[variavel_num] >= limite_inferior) & (df[variavel_num] <= limite_superior)]  # remove os outliers
    return df                                                    # retorna o dataframe com os outliers removidos

# Removendo os Outliers do Dataframe
df_sem_outliers = remover_outliers_iqr(df_churn, 'CS_MensalidadeAtual')

# Determinando a dimensão do Dataset após a remoção dos outliers
df_sem_outliers.shape

In [ ]:
df_sem_outliers.info()

In [ ]:
# checkpoint
# copiando o dataset para treinamento
df_treinamento = df_sem_outliers.copy()

# removendo as colunas desnecessárias
df_treinamento = df_treinamento.drop(columns=['PartnersId_x', 'Billing_PaganteAtivo','estadia_churn_meses', 'estadia_real_meses', 'PloomesId', 'Nome', 'DataCriacao_Date', 'ADM_DataAtivacao_Date', 'data_churn'])

# exibindo o dataset
df_treinamento.head()

### 04-02 <font color='#330870'><b> Separação dos dados de treinamento e teste

<font color='#330870'> Separamos o conjunto de dados de treinamento e teste com uma proporção de 80% de dados para treinamento e 20% para teste.

In [ ]:
from sklearn.model_selection import train_test_split

# Criar Dataset para treinamento através de uma cópia do dataset sem as variáveis categóricas que serão o atributo alvo
X = df_treinamento.drop(['churn'], axis=1).copy()

# Criar dataset com uma cópia do dataset original somente com o atributo alvo
y = df_treinamento['churn'].copy()

# Usando train_test_split para separar o conjunto de dados de teste e treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X.head(5)

In [ ]:
y.head(5)

<font color='#330870'> Para facilitar o pipeline de classificação, criamos também variáveis para identificar automaticamente as variáveis numéricas e categóricas do dataset de treinamento.

In [ ]:
# determinando as colunas categóricas e numéricas do dataset de treinamento
categorical = X_train.select_dtypes(include=['object']).columns.tolist()
numerical = X_train.select_dtypes(include=['number']).columns.tolist()

In [ ]:
# explicando os dados utilizados no treinamento

import pandas as pd
import numpy as np
import re
from datetime import datetime

# =========================
# CONFIG
# =========================
DF = X  # ajuste se seu dataframe tiver outro nome
N_EXAMPLES = 5
OUT_PREFIX = "data_dictionary"

# =========================
# Helpers
# =========================
def snake_to_title(s: str) -> str:
    s = re.sub(r"[_\-]+", " ", str(s)).strip()
    return re.sub(r"\s+", " ", s).title()

def infer_semantic_group(col: str) -> str:
    c = col.lower()

    rules = [
        ("Identificação do cliente", ["partnersid", "ploomesid", "accountid", "id", "nome", "label_cliente", "customer"]),
        ("Financeiro / Receita", ["mrr", "mensalidade", "billing", "receita", "arr", "cs_mensalidade"]),
        ("Engajamento (uso do produto)", ["engage", "users", "ativos", "tasks", "tarefas", "propostas", "deals", "moviment"]),
        ("Suporte / Produto", ["tickets", "chamados", "intercom", "jira", "issue"]),
        ("Satisfação / Voz do cliente", ["nps", "csat", "feedback", "mentionchurn", "churn", "satisf"]),
        ("Segmentação / Contexto do cliente", ["moc", "tipoimplementacao", "classificacao", "estagiojornada", "status"]),
        ("Datas / Marcos", ["data", "date", "dt", "mes"]),
        ("Scores / Labels (modelagem)", ["score", "risk", "label", "target", "y_"]),
    ]

    for group, keys in rules:
        if any(k in c for k in keys):
            return group
    return "Outros"

def infer_measure_type(col: str) -> str:
    c = col.lower()
    if "qoq" in c:
        return "Variação (QoQ / razão)"
    if "avg" in c or "mean" in c:
        return "Média"
    if "stdev" in c or "std" in c:
        return "Volatilidade (desvio padrão)"
    if "count" in c or "qtde" in c or "quant" in c:
        return "Contagem"
    if "pct" in c or "percent" in c:
        return "Percentual"
    if "mrr" in c or "mensalidade" in c or "billing" in c:
        return "Valor monetário"
    if "date" in c or "data" in c or "mes" in c:
        return "Data/Tempo"
    return "Atributo / métrica"

def pandas_dtype_label(series: pd.Series) -> str:
    dt = series.dtype
    if pd.api.types.is_integer_dtype(dt): return "inteiro"
    if pd.api.types.is_float_dtype(dt): return "decimal"
    if pd.api.types.is_bool_dtype(dt): return "booleano"
    if pd.api.types.is_datetime64_any_dtype(dt): return "data/hora"
    return "texto"

def try_parse_dates(s: pd.Series) -> bool:
    # tenta reconhecer colunas que são datas em texto
    if pd.api.types.is_datetime64_any_dtype(s.dtype):
        return True
    if s.dtype == "object":
        sample = s.dropna().astype(str).head(50)
        if sample.empty:
            return False
        parsed = pd.to_datetime(sample, errors="coerce", utc=False)
        return parsed.notna().mean() > 0.7
    return False

def summarize_numeric(s: pd.Series):
    s2 = pd.to_numeric(s, errors="coerce")
    if s2.dropna().empty:
        return {"min": None, "p50": None, "max": None}
    return {
        "min": float(np.nanmin(s2)),
        "p50": float(np.nanpercentile(s2, 50)),
        "max": float(np.nanmax(s2)),
    }

def safe_examples(s: pd.Series, n=N_EXAMPLES):
    vals = s.dropna().unique()
    vals = vals[:n]
    return ", ".join([str(v) for v in vals])

# =========================
# Build dictionary
# =========================
rows = []
for col in DF.columns:
    s = DF[col]
    non_null = int(s.notna().sum())
    total = int(len(s))
    missing_pct = round((1 - non_null / total) * 100, 2) if total else 0.0

    group = infer_semantic_group(col)
    measure_type = infer_measure_type(col)

    # tipo inferido (mais semântico do que dtype puro)
    semantic_dtype = "data/hora" if try_parse_dates(s) else pandas_dtype_label(s)

    # exemplos
    examples = safe_examples(s, N_EXAMPLES)

    # estatísticas se numérico
    num_stats = {"min": None, "p50": None, "max": None}
    if pd.api.types.is_numeric_dtype(s.dtype):
        num_stats = summarize_numeric(s)
    else:
        # tenta detectar numérico mesmo sendo texto (ex: "4.0")
        coerced = pd.to_numeric(s, errors="coerce")
        if coerced.notna().mean() > 0.8:
            num_stats = summarize_numeric(coerced)
            semantic_dtype = "decimal"

    # descrição automática curta
    # (você pode editar depois manualmente para uma versão “final”)
    description = (
        f"{snake_to_title(col)}. "
        f"Grupo: {group}. Tipo: {measure_type}. "
        f"Armazena um campo de {semantic_dtype} relacionado a {group.lower()}."
    )

    rows.append({
        "field_name": col,
        "field_title": snake_to_title(col),
        "group": group,
        "measure_type": measure_type,
        "data_type_inferred": semantic_dtype,
        "missing_%": missing_pct,
        "examples": examples,
        "min": num_stats["min"],
        "p50": num_stats["p50"],
        "max": num_stats["max"],
        "auto_description": description,
    })

dictionary_df = pd.DataFrame(rows).sort_values(["group", "field_name"]).reset_index(drop=True)

# =========================
# Export outputs
# =========================
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = f"{OUT_PREFIX}_{ts}.csv"
md_path = f"{OUT_PREFIX}_{ts}.md"

dictionary_df.to_csv(csv_path, index=False)

# Markdown simples para colar em docs
with open(md_path, "w", encoding="utf-8") as f:
    f.write("# Data Dictionary (gerado automaticamente)\n\n")
    f.write(dictionary_df.to_markdown(index=False))

print("Arquivos gerados:")
print(" -", csv_path)
print(" -", md_path)

dictionary_df.head(20)

### 04-03 <font color='#330870'><b> Construção do modelo

#### 04-03-01 Construindo o pipeline

In [ ]:
# importando o classificador RandomForest
from sklearn.ensemble import RandomForestClassifier

# importando o classificador DecisionTree
from sklearn.tree import DecisionTreeClassifier

<font color='#330870'> Definindo o pré-processador:

In [ ]:
# Bibliotecas de aprendizado de máquina
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


# criar variável que define quais pré-processamentos serão feitos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', TargetEncoder(), categorical)
    ])

<font color='#330870'> Definindo os pipelines classificadores para modelo:

In [ ]:
# criando um pipeline para cada classificador (DecisionTree, RandomForest e GaussianNaive Bayes)
classificador_dt = Pipeline(
    steps=[
    ('preprocessing', preprocessor),
    ('classifier', DecisionTreeClassifier())
    ]
    )

classificador_rf = Pipeline(
    steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
    ]
    )

#### 04-03-02 Treinando o modelo e fazendo as predições

<font color='#330870'> Após a definição do pipeline classificador, utilizamos os dados de treinamento, representados pelos dataset X_train e y_train, para realizar o treinamento do modelo.

##### <font color='#330870'> Classificador Decision Tree

<font color='#330870'> Treinando o modelo classificador com **Decision Tree**

In [ ]:
# Treinar o modelo supervisionado (ou seja, possuo a classe alvo em y) para cada classificador
classificador_dt.fit(X_train, y_train)

<font color='#330870'> Com o modelo classificador treinado, podemos então fazer as predições para os dados de teste e de treino.

In [ ]:
# Fazer as predições para os dados de teste
y_pred_test_dt = classificador_dt.predict(X_test)

# Fazer as predições para os dados de treino
y_pred_train_dt = classificador_dt.predict(X_train)

##### <font color='#330870'>Classificador Random Forest

<font color='#330870'> Treinando o modelo classificador com **Random Forest**

In [ ]:
# Treinar o modelo supervisionado (ou seja, possuo a classe alvo em y) para cada classificador
classificador_rf.fit(X_train, y_train)

<font color='#330870'> Com o modelo classificador treinado, podemos então fazer as predições para os dados de teste e de treino.

In [ ]:
# Fazer as predições para os dados de teste
y_pred_test_rf = classificador_rf.predict(X_test)

# Fazer as predições para os dados de treino
y_pred_train_rf = classificador_rf.predict(X_train)

### 04-04 <font color='#330870'><b> Avaliação da performance dos modelos

<font color='#330870'> Agora com o modelo treinado e as predições feitas, podemos avaliar a performance do modelo. Inicialmente faremos a avaliação com o ClassificationReport e a Matriz de Confusão, a fim de obter a acurácia do modelo, sua precissão e recall.

In [ ]:
# printando a avaliação classification report para cada classificador
print("Classification Report - Decision Tree:")
print(classification_report(y_test, y_pred_test_dt))
print("")
print("\nClassification Report - Random Forest:")
print(classification_report(y_test, y_pred_test_rf))

In [ ]:
# Gerando a matriz de confusão para cada modelo classificador
cm_dt = confusion_matrix(y_test, y_pred_test_dt)
cm_rf = confusion_matrix(y_test, y_pred_test_rf)

# plotando as matrizes de confusão
disp = ConfusionMatrixDisplay(confusion_matrix=cm_dt, display_labels=classificador_dt.classes_)
disp.plot(cmap='Purples')
plt.title('Matriz de Confusão - Decision Tree')
plt.show()

disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=classificador_rf.classes_)
disp.plot(cmap='Purples')
plt.title('Matriz de Confusão - Random Forest')
plt.show()

In [ ]:
# checando a proporção de y_test values para cada classificador

print("Proporção de y_test values para cada classificador:")
print("")
print("Decision Tree:", y_test.value_counts(normalize=True))
print("")
print("Random Forest:", y_test.value_counts(normalize=True))

### 04-05 <font color='#330870'><b> Validação do modelo

<font color='#330870'> Para fazer uma validação final do modelo, garantindo que ele é capaz de generalizar para novos dados e que não caiu em overfitting ou underfitting.

<font color='#330870'> A curva de aprendizado permite diagnosticar visualmente esse comportamento e tomar decisões sobre ajustes no modelo, como poda, ajuste de profundidade ou até uso de outros algoritmos mais robustos.

<font color='#330870'> Portanto, plotamos abaixo a curva de aprendizado dos modelos de classificação.

In [ ]:
# Calculando e plotando a curva de aprendizado para cada classificador
from sklearn.model_selection import learning_curve

In [ ]:
# calculando a curva de aprendizado para DecisionTree
train_sizes, train_scores, val_scores = learning_curve(
    classificador_dt,                         # modelo de classificação
    X,                                        # dados de treinamento
    y,                                        # dados alvo
    cv=10,                                    # validação cruzada com 10 folds
    scoring='accuracy',                       # métrica de avaliação
    n_jobs=-1,                                # número de núcleos da CPU utilizados (-1 = todos disponíveis)
    train_sizes=np.linspace(0.1, 1.0, 10)     # proporção dos conjuntos de treino
)

train_mean = train_scores.mean(axis=1)
test_mean = val_scores.mean(axis=1)

# Plotando
plt.figure(figsize=(10, 6))
plt.ylim(0, 1.1)

# Linha de treino
line1, = plt.plot(train_sizes, train_mean, marker='o', label="Treino", color='purple')
for x, y_val in zip(train_sizes, train_mean):
    plt.text(x, y_val + 0.02, f"{y_val:.2f}", color=line1.get_color(), fontsize=9, ha='center')

# Linha de validação
line2, = plt.plot(train_sizes, test_mean, marker='s', label="Validação", color='green')
for x, y_val in zip(train_sizes, test_mean):
    plt.text(x, y_val + 0.02, f"{y_val:.2f}", color=line2.get_color(), fontsize=9, ha='center')

plt.xlabel("Tamanho do conjunto de treino")
plt.ylabel("Acurácia")
plt.title("Curva de Aprendizado - Decision Tree")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# calculando a curva de aprendizado para RandomForest
train_sizes, train_scores, val_scores = learning_curve(
    classificador_rf,                         # modelo de classificação
    X,                                        # dados de treinamento
    y,                                        # dados alvo
    cv=10,                                    # validação cruzada com 10 folds
    scoring='accuracy',                       # métrica de avaliação
    n_jobs=-1,                                # número de núcleos da CPU utilizados (-1 = todos disponíveis)
    train_sizes=np.linspace(0.1, 1.0, 10)     # proporção dos conjuntos de treino
)

train_mean = train_scores.mean(axis=1)
test_mean = val_scores.mean(axis=1)

# Plotando
plt.figure(figsize=(10, 6))
plt.ylim(0, 1.1)

# Linha de treino
line1, = plt.plot(train_sizes, train_mean, marker='o', label="Treino", color='purple')
for x, y_val in zip(train_sizes, train_mean):
    plt.text(x, y_val + 0.02, f"{y_val:.2f}", color=line1.get_color(), fontsize=9, ha='center')

# Linha de validação
line2, = plt.plot(train_sizes, test_mean, marker='s', label="Validação", color='green')
for x, y_val in zip(train_sizes, test_mean):
    plt.text(x, y_val + 0.02, f"{y_val:.2f}", color=line2.get_color(), fontsize=9, ha='center')

plt.xlabel("Tamanho do conjunto de treino")
plt.ylabel("Acurácia")
plt.title("Curva de Aprendizado - RandomForest")
plt.legend()
plt.grid(True)
plt.show()

### 04-06~08 hiperparametrização, threshold, performance modelo hiperparametrizado ????

### 04-09  <font color='#330870'>  Explicabilidade do modelo

<font color='#330870'> A explicabilidade em modelos de machine learning é fundamental para garantir confiança, transparência e usabilidade prática, principalmente em contextos de tomada de decisão empresarial. Entender como o modelo toma suas decisões permite que usuários não técnicos validem se o raciocínio da máquina está alinhado com o conhecimento de negócio. Além disso, essa análise ajuda a identificar possíveis vieses, variáveis irrelevantes ou até oportunidades para melhorias no modelo. Técnicas como SHAP são especialmente úteis, pois avaliam o impacto individual de cada feature nas predições, ajudando a avaliar o comportamento do modelo tanto de forma agregada (importância média) quanto em instâncias específicas.

In [ ]:
# @title
import pandas as pd
import joblib
import shap
import matplotlib.pyplot as plt

# Carrega o modelo (pipeline completo)
modelo = classificador_rf  # seu pipeline

#copiando somente 10 registros do df_treinamento
#df_explica = df_treinamento.iloc[:10].copy()
df_explica = df_treinamento.copy()

# Obtém apenas as colunas usadas no modelo
colunas_utilizadas = modelo.feature_names_in_
df_modelo = df_explica[colunas_utilizadas]


# Supondo que seu Pipeline tenha steps nomeados como:
# 'preprocessing' => ColumnTransformer
# 'classifier'    => RandomForest ou outro modelo
preprocessador = modelo.named_steps["preprocessing"]
modelo_final = modelo.named_steps["classifier"]

# Aplica transformação
X_preprocessed = preprocessador.transform(df_modelo)

# Obtém os nomes corretos das colunas após transformação
feature_names = preprocessador.get_feature_names_out()

df_preprocessado = pd.DataFrame(X_preprocessed, columns=feature_names)


# --- Feature Importance (se disponível)
if hasattr(modelo_final, "feature_importances_"):
    importances = modelo_final.feature_importances_
    importance_df = pd.DataFrame({
        "Feature": feature_names,
        "Importance": importances
    }).sort_values(by="Importance", ascending=False)

    plt.figure(figsize=(10, 6))
    plt.barh(importance_df["Feature"], importance_df["Importance"], color="purple")
    plt.xlabel("Importância")
    plt.title("Importância das Features no Modelo")
    #exibindo os rótulos dos dados
    for i, v in enumerate(importance_df["Importance"]):
        plt.text(v, i, str(round(v, 3)), color='black', fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# --- SHAP
explainer = shap.Explainer(modelo_final.predict_proba, df_preprocessado)
shap_values = explainer(df_preprocessado)

# Selecionar índice da classe positiva (ex: 'True')
classes = modelo_final.classes_
print("Classes disponíveis:", classes)

# Ajuste aqui caso 'Ganha' esteja em outra posição
idx_classe_positiva = list(classes).index(True)

# Seleciona os shap_values apenas da classe positiva
shap_values_pos = shap_values[..., idx_classe_positiva]

# Gráfico SHAP de barras
shap.plots.bar(shap_values_pos, max_display=10)

# Gráfico SHAP beeswarm
shap.plots.beeswarm(shap_values_pos)